In [61]:
# Import necessary libraries
import alpaca
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt


# Set your API key and secret
API_KEY = 'PKPJWD9MRK2Z3T4L41CK'
API_SECRET = 'wkyHvvmfGRdMr9H57KddPrD8zwG3sQ0KwO4GlojT'

# Initialize the Alpaca data client
client = StockHistoricalDataClient(API_KEY, API_SECRET)

# Define the function to get historical data
def get_historical_data(symbol, start_date, end_date):
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame.Day,
        start=start_date,
        end=end_date
    )
    bars = client.get_stock_bars(request_params).df
    return bars


# Calculate RSI
def calculate_rsi(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calculate EMA
def calculate_ema(data, span=20):
    ema = data['close'].ewm(span=span, adjust=False).mean()
    return ema

# Calculate SMA
def calculate_sma(data, window=7):
    sma = data['close'].rolling(window=window).mean()
    return sma

# Calculate Bollinger Bands
def calculate_bollinger_bands(data, window=20):
    rolling_mean = data['close'].rolling(window=window).mean()
    rolling_std = data['close'].rolling(window=window).std()
    bollinger_upper = rolling_mean + (rolling_std * 2)
    bollinger_lower = rolling_mean - (rolling_std * 2)
    return bollinger_upper, rolling_mean, bollinger_lower

# Calculate MACD
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

# Calculate KD (Stochastic Oscillator)
def calculate_kd(data, window=14):
    low_min = data['low'].rolling(window=window).min()
    high_max = data['high'].rolling(window=window).max()
    k = 100 * (data['close'] - low_min) / (high_max - low_min)
    d = k.rolling(window=3).mean()
    return k, d


# Prepare data for LSTM model
def prepare_data_for_lstm(data, n_lookback):
    X, y = [], []
    for i in range(n_lookback, len(data)):
        X.append(data[i-n_lookback:i])
        y.append(data[i, -1])  # 'adjusted_close' price is the target
    return np.array(X), np.array(y)

# Define the model creation function
def create_model(units=100, dropout_rate=0.3, input_shape=(60, 18), activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(units=units, return_sequences=True, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Get today's date and the date one year ago
end_date = datetime.now() - timedelta(days=2)
start_date = end_date - timedelta(days=1500)

# Specify the ticker
ticker = 'AAPL'

# Fetch historical data for the specified ticker and calculate indicators
print(f"Fetching data for {ticker}")
try:
    data = get_historical_data(ticker, start_date, end_date)
    data['RSI'] = calculate_rsi(data)
    data['EMA_20'] = calculate_ema(data, span=20)
    data['EMA_50'] = calculate_ema(data, span=50)
    data['EMA_100'] = calculate_ema(data, span=100)
    data['EMA_200'] = calculate_ema(data, span=200)
    data['SMA_7'] = calculate_sma(data, window=7)
    data['SMA_21'] = calculate_sma(data, window=21)
    data['Bollinger_Upper'], data['Bollinger_Middle'], data['Bollinger_Lower'] = calculate_bollinger_bands(data)
    data['MACD'], data['MACD_Signal'] = calculate_macd(data)
    data['%K'], data['%D'] = calculate_kd(data)
    data = data.drop(columns=['trade_count', 'volume', 'vwap'])  # Drop the specified columns

    # Check and reset the index if necessary
    if 'date' in data.columns:
        data.set_index('date', inplace=True)
    data = data.reset_index()

    # Fill missing values
    data = data.fillna(method='ffill').fillna(method='bfill')

except Exception as e:
    print(f"Could not fetch data for {ticker}: {e}")

# Select relevant features for model input
X = ['open', 'high', 'low', 'close', 'RSI', 'EMA_20', 'EMA_50', 'EMA_100',
                   'EMA_200', 'SMA_7', 'SMA_21', 'Bollinger_Upper',
                   'Bollinger_Middle', 'Bollinger_Lower', 'MACD', 'MACD_Signal', '%K', '%D']
y = 'adjusted_close'


# Split the data into training and testing sets

train_size = int(len(data) * 0.8)
train_data = data[:train_size]
test_data = data[train_size:]

# Normalize the data

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data[X])
test_scaled = scaler.transform(test_data[X])

# Prepare the data for LSTM model

n_lookback = 60
X_train, y_train = prepare_data_for_lstm(train_scaled, n_lookback)
X_test, y_test = prepare_data_for_lstm(test_scaled, n_lookback)

# Create the LSTM model

model = create_model(units=100, dropout_rate=0.3, input_shape=(n_lookback, len(X)), activation='relu', optimizer='adam')

# Train the model

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Make predictions

train_predictions = model.predict(X_train)

test_predictions = model.predict(X_test)


Fetching data for AAPL


C:\Users\AI\AppData\Local\Temp\ipykernel_13204\3087421694.py:132: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill').fillna(method='bfill')


Epoch 1/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 12s 171ms/step - loss: 0.2142 - val_loss: 0.0619
Epoch 2/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - loss: 0.0623 - val_loss: 0.0463
Epoch 3/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - loss: 0.0546 - val_loss: 0.0401
Epoch 4/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - loss: 0.0445 - val_loss: 0.0275
Epoch 5/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - loss: 0.0346 - val_loss: 0.0227
Epoch 6/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 120ms/step - loss: 0.0284 - val_loss: 0.0188
Epoch 7/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 128ms/step - loss: 0.0257 - val_loss: 0.0165
Epoch 8/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - loss: 0.0275 - val_loss: 0.0186
Epoch 9/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - loss: 0.0263 - val_loss: 0.0137
Epoch 10/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - loss: 0.0233 - val_loss: 0.0115
Epoch 11/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step - loss: 0.0207 - val_loss: 0.0116
Epoch 12/100
24/24 ━━━━━━━━━━━━━━━━━━━━ 3

KeyError: 'date'

<Figure size 1400x700 with 0 Axes>